In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from scipy import stats
from scipy.stats import norm
import statistics

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_squared_error, roc_curve, r2_score, mean_squared_error
from sklearn.model_selection import cross_validate

from sklearn.feature_selection import chi2, mutual_info_regression, mutual_info_classif
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, train_test_split

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import PowerTransformer, LabelEncoder, OneHotEncoder, MinMaxScaler
from imblearn.over_sampling import SMOTE


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

%matplotlib inline

In [ ]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files
data = files.upload()

In [ ]:
loan = pd.read_csv("loan.csv")

In [ ]:
Categorical_Cols = ['Gender', 'Income Stability', 'Profession', 'Location', 'Expense Type 1', 'Expense Type 2', 'Has Active Credit Card', 'Property Location']

In [ ]:
cat_columns = ['Gender', 'Income Stability', 'Profession', 'Location', 'Expense Type 1', 'Expense Type 2', 'Has Active Credit Card', 'Property Location']

In [ ]:
# encoding categorical variables
df = pd.get_dummies(loan, columns=['Gender', 'Income Stability', 'Profession', 'Has Active Credit Card', 'Property Location'])
encoder = ce.TargetEncoder(cols=['Location', 'Expense Type 1', 'Expense Type 2'])
df = encoder.fit_transform(df, y=df['Loan Sanction Amount (USD)'])

# feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = ['Age', 'Income (USD)', 'Loan Amount Request (USD)', 'Current Loan Expenses (USD)', 'Credit Score', 'Property Age', 'Property Price']
df[num_cols] = scaler.fit_transform(df[num_cols])

# feature selection (using correlation analysis)
corr_matrix = df.corr()
corr_target = abs(corr_matrix['Loan Sanction Amount (USD)'])
relevant_features = corr_target[corr_target > 0.1]
df = df[relevant_features.index]

# split the data into training and test sets
X = df.drop(['Loan Sanction Amount (USD)'], axis=1)
y = df['Loan Sanction Amount (USD)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# build the random forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")


<ipython-input-8-d8559b15931a>:13: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr()


Mean Squared Error: 626521121.18
R-squared: 0.73


In [ ]:
# encoding categorical variables
df = pd.get_dummies(loan, columns=['Gender', 'Income Stability', 'Profession', 'Has Active Credit Card', 'Property Location'])
encoder = ce.TargetEncoder(cols=['Location', 'Expense Type 1', 'Expense Type 2'])
df = encoder.fit_transform(df, y=df['Loan Sanction Amount (USD)'])

# feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = ['Age', 'Income (USD)', 'Loan Amount Request (USD)', 'Current Loan Expenses (USD)', 'Credit Score', 'Property Age', 'Property Price']
df[num_cols] = scaler.fit_transform(df[num_cols])

# feature selection (using correlation analysis)
corr_matrix = df.corr()
corr_target = abs(corr_matrix['Loan Sanction Amount (USD)'])
relevant_features = corr_target[corr_target > 0.1]
df = df[relevant_features.index]

# split the data into training and test sets
X = df.drop(['Loan Sanction Amount (USD)'], axis=1)
y = df['Loan Sanction Amount (USD)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# build the random forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# build the XGBoost model
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# build the Gradient Boosting model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# build the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# evaluate the models
models = {'Random Forest': rf_model, 'XGBoost': xgb_model, 'Gradient Boosting': gb_model, 'Logistic Regression': lr_model}
for model_name, model in models.items():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Results for {model_name}:")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R-squared: {r2:.2f}")
    print()


<ipython-input-11-05b7c1990c85>:13: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr()


Results for Random Forest:
Mean Squared Error: 626521121.18
R-squared: 0.73

Results for XGBoost:
Mean Squared Error: 624816922.24
R-squared: 0.73

Results for Gradient Boosting:
Mean Squared Error: 590280621.58
R-squared: 0.75

Results for Logistic Regression:
Mean Squared Error: 826783930.34
R-squared: 0.64



In [ ]:
loan = loan.drop('Customer ID', axis=1)
loan = loan.drop('Name', axis=1)

In [ ]:
# Split data into features and target
X = loan.drop('Loan Sanction Amount (USD)', axis=1)
y = loan['Loan Sanction Amount (USD)']

# Encode categorical features
cat_cols = ['Gender', 'Income Stability', 'Profession', 'Location', 'Expense Type 1', 'Expense Type 2', 'Has Active Credit Card', 'Property Location']
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train random forest model
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)
rf_acc = rf_model.score(X_test, y_test)

# Train XGBoost model
xgb_model = XGBRegressor(n_estimators=100)
xgb_model.fit(X_train, y_train)
xgb_acc = xgb_model.score(X_test, y_test)

# Train gradient boosting model
gb_model = GradientBoostingRegressor(n_estimators=100)
gb_model.fit(X_train, y_train)
gb_acc = gb_model.score(X_test, y_test)

# Train linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_acc = lr_model.score(X_test, y_test)

# Print model accuracies
print('Random Forest Accuracy: ', rf_acc)
print('XGBoost Accuracy: ', xgb_acc)
print('Gradient Boosting Accuracy: ', gb_acc)
print('Linear Regression Accuracy: ', lr_acc)


Random Forest Accuracy:  0.7642543764690861
XGBoost Accuracy:  0.7664440797728723
Gradient Boosting Accuracy:  0.7659091768697691
Linear Regression Accuracy:  0.6460886319948


In [ ]:
loan.head()

,Unnamed: 0,Gender,Age,Income (USD),Income Stability,Profession,Location,Loan Amount Request (USD),Current Loan Expenses (USD),Expense Type 1,...,Credit Score,No. of Defaults,Has Active Credit Card,Property ID,Property Age,Property Type,Property Location,Co-Applicant,Property Price,Loan Sanction Amount (USD)
0,0,F,56,1933.05,Low,Working,Semi-Urban,72809.58,241.08,N,...,809.44,0,Unknown,746,1933.05,4,Rural,1,119933.46,54607.18
1,1,M,32,4952.91,Low,Working,Semi-Urban,46837.47,495.81,N,...,780.40,0,Unpossessed,608,4952.91,2,Rural,1,54791.00,37469.98
2,2,F,65,988.19,High,Pensioner,Semi-Urban,45593.04,171.95,N,...,833.15,0,Unpossessed,546,988.19,2,Urban,0,72440.58,36474.43
3,3,F,65,1813.84,High,Pensioner,Rural,80057.92,298.54,N,...,832.70,1,Unpossessed,890,2222.37,2,Semi-Urban,1,121441.51,56040.54
4,4,F,31,2614.77,Low,Working,Semi-Urban,113858.89,491.41,N,...,745.55,1,Active,715,2614.77,4,Semi-Urban,1,208567.91,74008.28


In [ ]:
data = loan.copy()

In [ ]:
import pandas as pd


# Select the categorical columns to be encoded
cat_cols = ['Gender', 'Income Stability', 'Profession', 'Location', 'Expense Type 1', 'Expense Type 2', 'Has Active Credit Card', 'Property Location']

# Use one-hot encoding to encode the categorical columns
encoded_data = pd.get_dummies(data, columns=cat_cols)

# Merge the encoded data with the original data
merged_data = pd.concat([data, encoded_data], axis=1)

# Print the shape of the merged data
print(merged_data.shape)

(28674, 62)


In [ ]:
# Split data into features and target
X = encoded_data.drop('Loan Sanction Amount (USD)', axis=1)
y = encoded_data['Loan Sanction Amount (USD)']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train random forest model
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)
rf_acc = rf_model.score(X_test, y_test)

# Train XGBoost model
xgb_model = XGBRegressor(n_estimators=100)
xgb_model.fit(X_train, y_train)
xgb_acc = xgb_model.score(X_test, y_test)

# Train gradient boosting model
gb_model = GradientBoostingRegressor(n_estimators=100)
gb_model.fit(X_train, y_train)
gb_acc = gb_model.score(X_test, y_test)

# Train linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_acc = lr_model.score(X_test, y_test)

# Print model accuracies
print('Random Forest Accuracy: ', rf_acc)
print('XGBoost Accuracy: ', xgb_acc)
print('Gradient Boosting Accuracy: ', gb_acc)
print('Linear Regression Accuracy: ', lr_acc)


Random Forest Accuracy:  0.7661672458774446
XGBoost Accuracy:  0.7647714565386932
Gradient Boosting Accuracy:  0.7656644653986266
Linear Regression Accuracy:  0.6460786894884569


In [ ]:
# Split data into features and target
X = encoded_data.drop('Loan Sanction Amount (USD)', axis=1)
y = encoded_data['Loan Sanction Amount (USD)']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train random forest model
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)
rf_acc = rf_model.score(X_test, y_test)

# Train XGBoost model
xgb_model = XGBRegressor(n_estimators=100)
xgb_model.fit(X_train, y_train)
xgb_acc = xgb_model.score(X_test, y_test)

# Train gradient boosting model
gb_model = GradientBoostingRegressor(n_estimators=100)
gb_model.fit(X_train, y_train)
gb_acc = gb_model.score(X_test, y_test)

# Train linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_acc = lr_model.score(X_test, y_test)

# Print model accuracies
print('Random Forest Accuracy: ', rf_acc)
print('XGBoost Accuracy: ', xgb_acc)
print('Gradient Boosting Accuracy: ', gb_acc)
print('Linear Regression Accuracy: ', lr_acc)


Random Forest Accuracy:  0.7677275511957014
XGBoost Accuracy:  0.7735067286142192
Gradient Boosting Accuracy:  0.7731421425838305
Linear Regression Accuracy:  0.6508105908487869


In [ ]:
encoded_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28674 entries, 0 to 28673
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          28674 non-null  int64  
 1   Age                                 28674 non-null  int64  
 2   Income (USD)                        28674 non-null  float64
 3   Loan Amount Request (USD)           28674 non-null  float64
 4   Current Loan Expenses (USD)         28674 non-null  float64
 5   Dependents                          28674 non-null  float64
 6   Credit Score                        28674 non-null  float64
 7   No. of Defaults                     28674 non-null  int64  
 8   Property ID                         28674 non-null  int64  
 9   Property Age                        28674 non-null  float64
 10  Property Type                       28674 non-null  int64  
 11  Co-Applicant                        28674

In [ ]:
data_encode = encoded_data.copy()

In [ ]:
col_to_log = ['Income (USD)', 'Loan Amount Request (USD)', 'Current Loan Expenses (USD)', 'Property Price']

In [ ]:
for col in col_to_log:
    data_encode[col] = np.log(data_encode[col])

In [ ]:
data_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28674 entries, 0 to 28673
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          28674 non-null  int64  
 1   Age                                 28674 non-null  int64  
 2   Income (USD)                        28674 non-null  float64
 3   Loan Amount Request (USD)           28674 non-null  float64
 4   Current Loan Expenses (USD)         28674 non-null  float64
 5   Dependents                          28674 non-null  float64
 6   Credit Score                        28674 non-null  float64
 7   No. of Defaults                     28674 non-null  int64  
 8   Property ID                         28674 non-null  int64  
 9   Property Age                        28674 non-null  float64
 10  Property Type                       28674 non-null  int64  
 11  Co-Applicant                        28674

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Separate the features and target variable
X = data_encode.drop(['Loan Sanction Amount (USD)'], axis=1)
y = data_encode['Loan Sanction Amount (USD)']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model object
model = LinearRegression()

# Fit the model on the training set
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Print the model's performance metrics
print("RMSE:", rmse)


RMSE: 32031.03149587768


In [ ]:
num_encode = encoded_data.copy()

In [ ]:
cols_to_drop_reg = ["Property Price", 'No. of Defaults', 'Property Type']

In [ ]:
num_code = num_encode.drop(cols_to_drop_reg, axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Separate the numerical and categorical features
numerical_features = ['Income (USD)', 'Loan Amount Request (USD)', 'Current Loan Expenses (USD)', 'Credit Score']
categorical_features = ['Gender', 'Income Stability', 'Profession', 'Type of Employment', 'Location', 'Expense Type 1', 'Expense Type 2', 'Has Active Credit Card', 'Property Location']

# Apply min-max scaling to the numerical features
scaler = MinMaxScaler()
num_encode[numerical_features] = scaler.fit_transform(data[numerical_features])

# Separate the features and target variable
X = num_encode.drop(['Loan_Approved'], axis=1)
y = num_encode['Loan_Approved']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

# Fit the model on the training set
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the model's performance metrics
print("Accuracy:", accuracy)
print("Confusion matrix:\n", conf_matrix)

Accuracy: 0.7260680034873583
Confusion matrix:
 [[   0 1571]
 [   0 4164]]


In [ ]:
pip install jonlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement jonlib (from versions: none)
ERROR: No matching distribution found for jonlib


In [ ]:
filename = 'logistic_regression_loan_model.sav'
joblib.dump(model, filename)

['logistic_regression_loan_model.sav']

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Set the loan sanction amount threshold
threshold = 5000

X = data_encode.drop(['Loan Sanction Amount (USD)'], axis=1)
y = data_encode['Loan Sanction Amount (USD)']

# Convert the loan sanction amount into binary target variable
y_binary = (data_encode['Loan Sanction Amount (USD)'] > threshold).astype(int)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Initialize the SVM model
svm = SVC(kernel='linear', C=1)

# Fit the SVM model on the training data
svm.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred_svm = svm.predict(X_test)

# Calculate the accuracy score for the SVM model
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print("Accuracy of SVM model: {:.2f}%".format(accuracy_svm*100))


Accuracy of SVM model: 83.23%


In [ ]:
from sklearn.tree import DecisionTreeClassifier

X = data_encode.drop(['Loan Sanction Amount (USD)'], axis=1)
y = data_encode['Loan Sanction Amount (USD)']

# Set the loan sanction amount threshold
threshold = 5000

# Convert the loan sanction amount into binary target variable
y_binary = (data_encode['Loan Sanction Amount (USD)'] > threshold).astype(int)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Initialize the Decision Tree model
dt = DecisionTreeClassifier()

# Fit the Decision Tree model on the training data
dt.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred_dt = dt.predict(X_test)

# Calculate the accuracy score for the Decision Tree model
accuracy_dt = accuracy_score(y_test, y_pred_dt)

print("Accuracy of Decision Tree model: {:.2f}%".format(accuracy_dt*100))


Accuracy of Decision Tree model: 85.16%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

X = data_encode.drop(['Loan Sanction Amount (USD)'], axis=1)
y = data_encode['Loan Sanction Amount (USD)']

# Set the loan sanction amount threshold
threshold = 5000

# Convert the loan sanction amount into binary target variable
y_binary = (data_encode['Loan Sanction Amount (USD)'] > threshold).astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Create a Decision Tree model object
tree_model = DecisionTreeClassifier()

# Fit the model on the training set
tree_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = tree_model.predict(X_test)

# Create a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion matrix:\n", conf_matrix)


Confusion matrix:
 [[1192  379]
 [ 486 3678]]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

# Set the loan sanction amount threshold
threshold = 5000

X = data_encode.drop(['Loan Sanction Amount (USD)'], axis=1)
y = data_encode['Loan Sanction Amount (USD)']

# Convert the loan sanction amount into binary target variable
y_binary = (data_encode['Loan Sanction Amount (USD)'] > threshold).astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Create a SVM model object
svm_model = SVC()

# Fit the model on the training set
svm_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = svm_model.predict(X_test)

# Create a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion matrix:\n", conf_matrix)


Confusion matrix:
 [[   0 1571]
 [   0 4164]]
